In [2]:
import sys, os, io, json
from neo4j import GraphDatabase
from py2neo import Graph
from pathlib import Path
from pandas import DataFrame
import pandas as pd
import networkx as nx

graph = Graph("bolt://localhost:7687")
driver = GraphDatabase.driver('bolt://localhost:7687', auth=None)

## For a set of chemicals, what are the associated genes and disease associations?




In [11]:
def chemical_to_diseases(chem_id):
    return DataFrame(graph.run("""
MATCH (Id { id: $id })<-[:id]-(chemical:GraphNode)-[r1:`biolink:affects`]->(affected:`biolink:Gene`)<-[r2:`ro:causally_related_to`]-(disease:GraphNode)
RETURN chemical.`grebi:name`[0] as chemical, affected.`grebi:name`[0] as affected_entity, disease.`grebi:name`[0] as disease, r1.`grebi:datasources` as affected_source, r2.`grebi:datasources` as cause_source
""", { 'id': chem_id }).data())

df = pd.concat([
    chemical_to_diseases('chebi:6801'), # Metformin,
    chemical_to_diseases('chebi:132447'), # PFOS
    chemical_to_diseases('chebi:91275'), # BBN
    chemical_to_diseases('chebi:5063') # Fipronil
], ignore_index=True)

print(df.to_markdown())

with open('chemicals_and_diseases.csv', 'w') as f:
    df.to_csv(f, index=False)


|      | chemical                                                                                                    | affected_entity         | disease                                                                                                             | affected_source         | cause_source   |
|-----:|:------------------------------------------------------------------------------------------------------------|:------------------------|:--------------------------------------------------------------------------------------------------------------------|:------------------------|:---------------|
|    0 | Metformin                                                                                                   | CTD Gene:2335 FN1       | spondylometaphyseal dysplasia corner fracture type                                                                  | ['Robokop.TextMining']  | ['UberGraph']  |
|    1 | Metformin                                                                